# Boston 房价预测

In [1]:
from keras.datasets import boston_housing
from keras import models
from keras import layers
import numpy as np

Using TensorFlow backend.


In [3]:
# 数据集下载
(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

In [4]:
print(train_data.shape, test_data.shape)

(404, 13) (102, 13)


In [5]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [6]:
# 数据处理
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

In [7]:
# 模型构建
def build_models():
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    
    return model

In [9]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #',i)
    val_data = train_data[i*num_val_samples : (i+1)*num_val_samples] # 划分出验证集部分
    val_targets = train_targets[i*num_val_samples : (i+1)*num_val_samples]

    partial_train_data = np.concatenate([train_data[:i*num_val_samples],train_data[(i+1)* num_val_samples:] ],axis=0) # 将训练集拼接到一起
    partial_train_targets = np.concatenate([train_targets[:i*num_val_samples],train_targets[(i+1)* num_val_samples:] ],axis=0)

    model = build_models()
    model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,batch_size=16,verbose=0)#模型训练silent模型
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0) # 验证集上评估
    all_scores.append(val_mae)


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [10]:
all_scores

[1.9035061727655995, 2.431620779604015, 2.5146334171295166, 2.370800970804573]

In [11]:
np.mean(all_scores)

2.305140335075926

In [13]:
model = build_models()
model.fit(train_data,train_targets,epochs=80,batch_size=16,verbose=0)
mes,mae = model.evaluate(test_data,test_targets)

102/102 [==============================] - 0s 386us/step


In [14]:
mes

18.02288085339116

In [15]:
mae

2.5730022729611863